<img src="images/Porter-logo.png" alt="Drawing" align="right" style="width: 100px;"/>

# Customer Lifetime Value


In this Notebook probabilistic models would be used to estimate CLV of Porter's customer base. The objective is to come up with a baseline CLV model which can be used a reference for comparing advanced ML models.
For building these models transactional data from completed_spot_orders_mv table is used

---


In [570]:
# Importing the libraries

import pandas as pd
from datetime import datetime as dt
import lifetimes as lt
from sklearn import metrics
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import random
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()


In [9]:
# Loading the data

orders=pd.read_csv('CLV/orders_1619.csv') # Time period : 2016-01-01 to 2019-12-31
orders['order_date']=pd.to_datetime(orders['order_date'],format='%Y-%m-%d')

### Data Pre processing

* Removing Transacrtion with negative customer fare
* Removing Fraudulent orders


In [10]:
# Removing transaction with negative customer fare
orders=orders[orders.customer_fare>0]

**Fraudulent orders**

It is possible that customer can do fraudulent transactions, to identify these we can plot the distribution of orders placed by customer in a day and find the outlier threshold.

The threshold is set to 5

In [11]:
df=orders[orders.order_date> '2019-01-01'] # Taking most recent year
orders_count=df.groupby(['customer_id','order_date'])['order_id'].count()
(orders_count.value_counts()/orders_count.shape[0])*100


1     88.863856
2      9.108297
3      1.505337
4      0.357182
5      0.105231
6      0.035664
7      0.015003
8      0.003772
9      0.002389
10     0.002053
11     0.000545
12     0.000377
13     0.000126
14     0.000084
20     0.000042
15     0.000042
Name: order_id, dtype: float64

In [12]:
# Removing the fraudulent orders
orders['order_count']=orders.groupby(['customer_id','order_date'])['order_id'].transform('count')
orders=orders[orders.order_count<6] 
orders=orders.drop('order_count',axis=1)

### Creating customer cohorts  

Since we have large customer base we will segment it into various cohorts and rules are based on date of first purchase and city. Currently logic for segementing customers are soley on business intuition

In [13]:
def create_cohort(data,city,end_date,start_date=None):
    if start_date is None:
        start_date=min(data['order_date'])
    
    temp_df=data.copy()
    temp_df['first_purchase_dt']=temp_df.groupby('customer_id')['order_date'].transform('min')
    cohort=temp_df.loc[(temp_df.first_purchase_dt.between(start_date,end_date)) & (temp_df.geo_region_id==city),]
    
    #Aggregating at a day level
    cohort=cohort.groupby(['customer_id','order_date'])['customer_fare'].sum().reset_index()
    
    print('Number of customers present: {}'.format(cohort.customer_id.nunique()))
    print('Number of orders: {}'.format(cohort.shape[0]))
    
    return(cohort)
    

In [234]:
# Selecting a particular cohort, city : Mumbai and first purchase between 2016-01 to 2016-06
cohort1=create_cohort(orders,1,end_date='2016-06-30')

Number of customers present: 16784
Number of orders: 379672


### Preprocessing cohort data

Each customer present in the cohort should be regular customer, in orders to define a regular customer following rules are followed :

* Customer should place atleast minimum number of orders in observation time
* Customer should not churn in the observation time period

Let's set the observeration time period for two years and preproces the cohort data accrodingly


**Minimum number of orders**

To find the minimum number of orders for each customer in the cohort, we will use the approach below :

*min_order=Cohort purchase rate*

*Cohort purchase rate = Total orders/Total customers*

In [259]:
def cohort_minorder(data):
    temp_df=data.copy()
    val=temp_df.shape[0]/temp_df.customer_id.nunique()
    print('minimum orders choosen for cohort:{}'.format(round(val,2)))
    temp_df['total_orders']=temp_df.groupby('customer_id')['order_date'].transform('count')
    temp_df=temp_df.loc[temp_df.total_orders >= val,]
    temp_df.drop('total_orders',axis=1,inplace=True)
    return temp_df


**Finding customers who have churned**

Before building the CLV model we should assure that none of the customers has already churned within the observation time period. In order to find the churn customers we can fit emperical cumulative ditribution function to cohort's between purchase time distribution and then use various percenetile value of this distribution to determine if the customer has churned already.

Please go through the following link for further explaination :

__[Link to Article](https://towardsdatascience.com/modelling-customer-churn-when-churns-are-not-explicitly-observed-with-r-a768a1c919d5)__



In [270]:
#Custom function for ecdf

def emperical_cdf(data,threshold):
    data=np.unique(data)
    percentiles=[]
    n=len(data)
    sort_data=np.sort(data)
    
    for i in np.arange(1,n+1):
        p=1-(i/n)
        percentiles.append(p)
    
    ecdf=pd.DataFrame({'timediff':sort_data,'prob':percentiles})
    return max(ecdf.loc[ecdf.prob>=threshold,'timediff'])



def cohort_churn_customer(data,end_date,threshold):
    T=dt.strptime(end_date,'%Y-%m-%d')
    temp_df=data.copy()
    temp_df=temp_df.sort_values(by=['customer_id','order_date'])
    temp_df['timediff']=temp_df.groupby('customer_id')['order_date'].diff(periods=1)
    temp_df['timediff']=temp_df.timediff.astype('timedelta64[D]')
    temp_df=temp_df.dropna()
    val=emperical_cdf(temp_df.timediff,threshold)
    print('Maximum days of gap choosen for cohort : {}'.format(val))
    temp_df['last_purchase_date']=temp_df.groupby('customer_id')['order_date'].transform('max')
    temp_df['last_order_gap']=T-temp_df.last_purchase_date
    temp_df['last_order_gap']=temp_df.last_order_gap.astype('timedelta64[D]')
    temp_df['max_timegap']=temp_df.groupby('customer_id')['timediff'].transform('max')
    temp_df=temp_df.loc[(temp_df.last_order_gap >= val) | (temp_df.max_timegap >= val)]
    customer_ids=np.unique(temp_df.customer_id)
    
    return customer_ids
        
    

In [271]:
# Combining the cohort_churn_customer and cohort_minorder functions

def cohort_preprocess(cohort,start_date,end_date,threshold=0.80):
    temp_df=cohort.copy()
    #Filtering data by observation time period
    temp_df=temp_df.loc[temp_df.order_date.between(start_date,end_date),]
    #Finding minorder value
    temp_df=cohort_minorder(temp_df)
    
    #Finding churn customers
    customer_ids=cohort_churn_customer(temp_df,end_date,threshold)
    #Removing churned customers
    final=temp_df.loc[~temp_df.customer_id.isin(customer_ids),]
    
    print('Number of customers present: {}'.format(final.customer_id.nunique()))
    print('Number of orders: {}'.format(final.shape[0]))
    #print('Number of customers removed after preprocess: {}'.format(a-final.customer_id.nunique()))
    
    return final
    
    

In [273]:
cohort1p=cohort_preprocess(cohort1,'2016-01-01','2017-12-31',threshold=0.70)

minimum orders choosen for cohort:13.6
Maximum days of gap choosen for cohort : 133.0
Number of customers present: 1443
Number of orders: 117560


### Exploratory analysis for the selected cohort 

In [479]:
df=cohort1p.groupby(cohort1p['order_date'].dt.strftime('%Y-%m'))['customer_fare'].sum().reset_index()
df['cumulative_revenue']=df['customer_fare'].cumsum()
fig = px.line(x=df['order_date'], y=df['cumulative_revenue'], labels={'x':'Time', 'y':'Cumulative Revenue'})
fig.add_shape(dict(type="line",x0='2017-01',y0=0,x1='2017-01',y1=80000000,line=dict(color="Red",width=1)))
fig.show()

As expeced the cumulative revenue of the cohort is increasing month over month in our observation and our goal is to predict this cumulative revenue after a certain point of time. For the time being Jan 2017 is choosen but it can change afterwards depending upon accuracy of model.

Let's try to visualize the cumulative revenue at an individual customer level :

In [566]:
#Filter random sample of 50 customers from the cohort

customer_ids=random.sample(np.unique(cohort1p.customer_id).tolist(),50)
df=cohort1p.loc[cohort1p.customer_id.isin(customer_ids),].groupby(['customer_id',cohort1p['order_date'].dt.strftime('%Y-%m')])['customer_fare'].sum().reset_index()
df['cumulative_revenue']=df.groupby('customer_id')['customer_fare'].cumsum()
fig = px.line(x=df['order_date'], y=df['cumulative_revenue'],color=df['customer_id'].astype(str),
              labels={'x':'Time', 'y':'Cumulative Revenue'})
fig.update_layout(legend_title='<b> Customer_ids </b>')
fig.show()

We can see there is hetrogeneity in each customer's purchasing behaviour, which we have to account for and probabilistc models account for this heterogenity by assuming some prior distributions for the parameters

### Model Development

Model development will include the following steps :

* Summarizing the Transaction data into RFM data
* Splitting summary data into calibration and holdout set
* Model Train
* Parameter tuning
* Model Evaluation

In [431]:
# Summarizing the data and splitting into calibration and holdout


def summarize_cohort_split(data,calibration_end,freq=7,observation_end=None):
    if observation_end is None:
        observation_end=data['order_date'].max()
    else:
        observation_end=dt.strptime(observation_end,'%Y-%m-%d')
    #finding number of periods in calibration and holdout data
    
    count_periods_c=(dt.strptime(calibration_end,'%Y-%m-%d')-data.order_date.min()).days
    count_periods_c=int(count_periods_c/freq)
    count_periods_h=(observation_end-dt.strptime(calibration_end,'%Y-%m-%d')).days
    count_periods_h=int(count_periods_h/freq)
    
    
    calibration_data=lifetimes.utils.summary_data_from_transaction_data(data,customer_id_col='customer_id',
                                            datetime_col='order_date',observation_period_end=calibration_end,
                                            datetime_format='%Y-%m-%d',freq='D',monetary_value_col='customer_fare',
                                                                        freq_multiplier=freq)
    
    
    
    temp_df=data.copy()
    temp_df=temp_df.loc[(temp_df.order_date > calibration_end) & (temp_df.order_date <=observation_end),]
    holdout_data=temp_df.groupby('customer_id')['order_date'].count().reset_index()
    holdout_data.rename(columns={'order_date':'actual'},inplace=True)
    print('shape of calibration data {}'.format(calibration_data.shape))
    print('shape of holdout data {}'.format(holdout_data.shape))
    print('periods in train and test {} {}'.format(count_periods_c,count_periods_h))
    
    return calibration_data,holdout_data,count_periods_c,count_periods_h

In [544]:
#Splitting preprocess cohort data into calibration and holdout
c,h,m,n=summarize_cohort_split(cohort1p,calibration_end='2016-12-31')

shape of calibration data (1443, 4)
shape of holdout data (1443, 2)
periods in train and test 52 52


**Model Train**

Follwoing probabilistic models are available in lifetime package :

* <strike>BG/BB</strike>
* BG/NBD
* MBG/NBD
* Pareto/NBD

_Note : All the above models predicts exprected orders not clv, to get clv we need expected revenue and for this we can try another model on top of these or use average monetary value_


In [619]:
def models_evaluate_holdout(model_obj,count_future_time_periods,hist_freq,hist_recency,hist_T,holdoutdata):
    val=model_obj.conditional_expected_number_of_purchases_up_to_time(count_future_time_periods,hist_freq,hist_recency,hist_T)
    return metrics.mean_absolute_error(holdoutdata['actual'],val)


def models_train(train,test,m,n,alpha=0.01):
    
    #Initilaizing the models instances
    #bg=lt.fitters.beta_geo_beta_binom_fitter.BetaGeoBetaBinomFitter(penalizer_coef=alpha)
    bgnbd=lt.fitters.beta_geo_fitter.BetaGeoFitter(penalizer_coef=alpha)
    mbg=lt.fitters.modified_beta_geo_fitter.ModifiedBetaGeoFitter(penalizer_coef=alpha)
    pareto=lt.fitters.pareto_nbd_fitter.ParetoNBDFitter(penalizer_coef=alpha)
    
    #Fitting in the training data
    #bg.fit(train['frequency'],train['recency'],train['T'])
    bgnbd.fit(train['frequency'],train['recency'],train['T'])
    mbg.fit(train['frequency'],train['recency'],train['T'])
    pareto.fit(train['frequency'],train['recency'],train['T'])
    
    #Out sample performance
    #mae_bgbb=models_evaluate_holdout(bg,n,train['frequency'],train['recency'],train['T'],test)
    mae_bgnbd=models_evaluate_holdout(bgnbd,n,train['frequency'],train['recency'],train['T'],test)
    mae_mbg=models_evaluate_holdout(mbg,n,train['frequency'],train['recency'],train['T'],test)
    mae_pareto=models_evaluate_holdout(pareto,n,train['frequency'],train['recency'],train['T'],test)
    
    #print('accuracy of bgbb model in holdout set {}'.format(mae_bgbb))
    print('MAE of bgnbd model in holdout set {}'.format(mae_bgnbd))
    print('MAE of mbg model in holdout set {}'.format(mae_mbg))
    print('MAE of pareto model in holdout set {}'.format(mae_pareto))
    
    return pareto

In [620]:
best_model=models_train(c,h,m,n,alpha=0.2)

MAE of bgnbd model in holdout set 19.38253428152132
MAE of mbg model in holdout set 19.381276879856053
MAE of pareto model in holdout set 18.954758846107925


**Model Performance at an cohort level**

In the below diagonistic plot we will visualize how these model are able to predict cumulative revenue and orders per week for the selcted cohort.

_Note : In order to convert predicted orders to revenue heuristic monetary value is used_

In [609]:
# Summarizing the holdout out set in weekly level

def custom_summarizer(data,start_date,end_date,freq=7):
    temp_df=data.copy()
    temp_df=temp_df.loc[temp_df.order_date.between(start_date,end_date),]
    temp_df=temp_df.groupby(temp_df['order_date'].dt.week).agg(customer_fare=('customer_fare','sum'),
                                                               orders=('order_date','count')).reset_index()
    temp_df['cumulative_revenue']=temp_df.customer_fare.cumsum()
    temp_df['cumulative_orders']=temp_df.orders.cumsum()
    return temp_df

# Predciting at weekly level

def estimate_values(model,future_periods,data):
    clv=[]
    orders_pred=[]
    for i in range(1,future_periods+1):
        orders=model.conditional_expected_number_of_purchases_up_to_time(i,data['frequency'],data['recency'],data['T']) 
        orders=np.array([int(i) for i in orders]).reshape(1,data.shape[0])
        monetary_value=np.array(c.monetary_value).reshape(data.shape[0],1)
        revenue=np.dot(orders,monetary_value)
        clv.append(revenue[0])
        orders_pred.append(np.sum(orders))
    return orders_pred,clv

In [612]:
# Predictions at weekly level
results_df=custom_summarizer(cohort1p,'2017-01-01','2017-12-31')
orders_predicted,clv_predicted=estimate_values(best_model,52,c)
results_df['clv_predicted']=np.vstack(clv_predicted)
results_df['orders_predicted']=orders_predicted

#Calculating percentage error
results_df['APE_orders']=abs(results_df['cumulative_orders']-results_df['orders_predicted'])/results_df['cumulative_orders']
results_df['APE_clv']=abs(results_df['cumulative_revenue']-results_df['clv_predicted'])/results_df['cumulative_revenue']


In [613]:
results_df

,order_date,customer_fare,orders,cumulative_revenue,cumulative_orders,clv_predicted,orders_predicted,APE_orders,APE_clv
0,1,671645.0,1100,671645.0,1100,3.308043e+05,530,0.518182,0.507471
1,2,681185.0,1140,1352830.0,2240,9.739177e+05,1621,0.276339,0.280089
2,3,748329.0,1159,2101159.0,3399,1.651809e+06,2786,0.180347,0.213858
3,4,663108.0,1024,2764267.0,4423,2.360214e+06,3989,0.098123,0.146170
4,5,739849.0,1225,3504116.0,5648,3.050872e+06,5156,0.087110,0.129346
5,6,766951.0,1204,4271067.0,6852,3.716685e+06,6303,0.080123,0.129799
6,7,742150.0,1206,5013217.0,8058,4.417404e+06,7494,0.069993,0.118848
7,8,692214.0,1133,5705431.0,9191,5.098460e+06,8662,0.057556,0.106385
8,9,804355.0,1323,6509786.0,10514,5.779358e+06,9819,0.066102,0.112205
9,10,791360.0,1278,7301146.0,11792,6.472738e+06,10992,0.067843,0.113463


In [615]:
# Visual plots
fig = go.Figure()
fig.add_trace(go.Scatter(x=results_df['order_date'], y=results_df['cumulative_revenue'],mode='lines',name='Actual'))

fig.add_trace(go.Scatter(x=results_df['order_date'], y=results_df['clv_predicted'],
                    mode='lines',
                    name=' Model'))

fig.update_layout(title= 'Model perfromance in predicting CLV',
    xaxis_title='Time (2017 onwards)',
                   yaxis_title='Cumulative revenue')
fig.show()



In [616]:
# Visual plots
fig = go.Figure()
fig.add_trace(go.Scatter(x=results_df['order_date'], y=results_df['cumulative_orders'],mode='lines',name='Actual'))

fig.add_trace(go.Scatter(x=results_df['order_date'], y=results_df['orders_predicted'],
                    mode='lines',
                    name=' Model'))

fig.update_layout(title= 'Model perfromance in predicting orders',
    xaxis_title='Time (2017 onwards)',
                   yaxis_title='Cumulative orders')
fig.show()



After looking these diagonistic plots it seem like our model is able to predict cumulative orders much better than cumulative revenue which suggest that using heuristic monetary value is not useful and we have to use to model based approach to estimate the CLV.